In [24]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import math
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or '3' to suppress all messages
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score
label_encoder = LabelEncoder()
pd.options.mode.chained_assignment = None


In [3]:
raw_data_path = './data/prive_24h_data.csv'
flag_data_path = './data/ground_truth_11_18.csv'

data_df = pd.read_csv(raw_data_path)
flag_data_df = pd.read_csv(flag_data_path)
data_df['gender'] = label_encoder.fit_transform(data_df['gender'])
data_df['race'] = label_encoder.fit_transform(data_df['race'])
data_df['admission_type_x'] = label_encoder.fit_transform(data_df['admission_type_x'])
data_df['first_careunit'] = label_encoder.fit_transform(data_df['first_careunit'])
data_df['admission_type_y'] = label_encoder.fit_transform(data_df['admission_type_y'])
data_df['insurance'] = label_encoder.fit_transform(data_df['insurance'])

In [4]:
def check_missing_values(df):
    if df.isna().any().any():
        return 1 
    else:
        return 0 

In [12]:
total_x = []
total_y = []
for index, row in flag_data_df.iterrows():
    id_df = data_df[data_df['stay_id'] == row['stay_id']]
    if not check_missing_values(id_df):
        label = id_df['label'].iloc[0] 
        id_df = id_df.drop(columns='label')
        id_df = id_df.drop(columns='charttime')
        total_x.append(id_df.values)
        total_y.append(label)

total_x = np.array(total_x)
total_y = np.array(total_y).reshape(-1, 1)

print(total_x.shape)
print(total_y.shape)
        

(2811, 24, 25)
(2811, 1)


In [18]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

def train_module(X_train, y_train, X_test, y_test, epoch, learning_rate, batch):

    # Define model and train
    def build_lstm_model(timesteps, num_features):
        model = tf.keras.Sequential([
            tf.keras.layers.LSTM(128, input_shape=(timesteps, num_features)),
            tf.keras.layers.Dense(1, activation='sigmoid') 
        ])
        model.build(input_shape=(None, timesteps, num_features))
        return model

    model = build_lstm_model(timesteps=X_train.shape[1], num_features=X_train.shape[2])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',  
                  metrics=['accuracy'])  

    model.fit(X_train, y_train, epochs=epoch, batch_size=batch, validation_data=(X_test, y_test))
    return model



In [25]:
total_x = total_x.astype(np.float32)
total_y = total_y.astype(np.float32)
total_x[np.isinf(total_x)] = np.nan
total_x[np.abs(total_x) > 1e6] = np.nan
# Handle NaN values by replacing them with zeros (you can choose a different strategy)
total_x[np.isnan(total_x)] = 0.0
scaler = MinMaxScaler()
total_x_normalized = scaler.fit_transform(total_x.reshape(-1, total_x.shape[-1])).reshape(total_x.shape)

X_train, X_test, y_train, y_test = train_test_split(total_x_normalized, total_y, test_size=0.2, random_state=42)
model = train_module(X_train, y_train, X_test, y_test, epoch=50, learning_rate=0.001, batch=32)
evaluation_results = model.evaluate(X_train, y_train)
print("Evaluation results:", evaluation_results)

Epoch 1/50


2023-11-21 05:52:27.090743: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-11-21 05:52:27.200374: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-11-21 05:52:27.277444: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


71/71 [==============================] - ETA: 0s - loss: 0.6529 - accuracy: 0.6397

2023-11-21 05:52:28.286262: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-11-21 05:52:28.330049: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


71/71 [==============================] - 2s 17ms/step - loss: 0.6529 - accuracy: 0.6397 - val_loss: 0.6621 - val_accuracy: 0.6146
Epoch 2/50
71/71 [==============================] - 1s 13ms/step - loss: 0.6241 - accuracy: 0.6739 - val_loss: 0.6409 - val_accuracy: 0.6412
Epoch 3/50
71/71 [==============================] - 1s 12ms/step - loss: 0.6254 - accuracy: 0.6557 - val_loss: 0.6392 - val_accuracy: 0.6359
Epoch 4/50
71/71 [==============================] - 1s 12ms/step - loss: 0.6085 - accuracy: 0.6770 - val_loss: 0.6364 - val_accuracy: 0.6448
Epoch 5/50
71/71 [==============================] - 1s 12ms/step - loss: 0.6034 - accuracy: 0.6886 - val_loss: 0.6470 - val_accuracy: 0.6394
Epoch 6/50
71/71 [==============================] - 1s 12ms/step - loss: 0.6088 - accuracy: 0.6828 - val_loss: 0.6229 - val_accuracy: 0.6590
Epoch 7/50
71/71 [==============================] - 1s 13ms/step - loss: 0.6036 - accuracy: 0.6846 - val_loss: 0.6356 - val_accuracy: 0.6501
Epoch 8/50
71/71 [======

In [26]:
evaluation_results = model.evaluate(X_test, y_test)
print("Evaluation results:", evaluation_results)

18/18 [==============================] - 0s 6ms/step - loss: 0.7422 - accuracy: 0.6234
Evaluation results: [0.7421980500221252, 0.6234458684921265]


In [28]:
y_pred_proba = model.predict(X_train)

# Calculate AUROC
auroc = roc_auc_score(y_train, y_pred_proba)
print("AUROC:", auroc)

71/71 [==============================] - 0s 5ms/step
AUROC: 0.7934060853073264
